In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.optim as optim

import math

torch.set_printoptions(sci_mode = False)
np.set_printoptions(suppress = True)

In [2]:
# grid discretisation over domain [0, 1] for x1 and x2
N_grid_x1 = 20 + 1 # 21 so steps are easy numbers
N_grid_x2 = N_grid_x1

N_grid = N_grid_x1 * N_grid_x2

x1 = np.linspace(0, 1, N_grid_x1) # x
x2 = np.linspace(0, 1, N_grid_x2) # y

# define distances
dx = 1 / (N_grid_x1 - 1)
dy = 1 / (N_grid_x2 - 1)

X1_test, X2_test = np.meshgrid(x1, x2)

## To Do: normalise?

This will of course impact the output scalar sigma_f

In [3]:
def simulate_convergence(X1, X2):
    U = X2
    V = X1
    return U, V

def simulate_merge(X1, X2):
    U = (X2 + 0.5)**2
    V = np.sin(X1 * math.pi)
    return U, V

def simulate_branching(X1, X2):
    U = X1 * X2
    V = - 0.5 * X2**2 + (X1 - 0.8)
    return U, V

def simulate_deflection(X1, X2):
    U = (X2 * 6 - 3)**2 + (X1 * 6 - 3)**2 + 3
    V = -2 * (X2 * 6 - 3) * (X1 * 6 - 3)
    return U, V

def simulate_ridge(X1, X2):
    U = X2 + 1
    V = - np.cos(3 * X1**3 * math.pi)
    return U, V

In [4]:
# flat [N, 1] arrays - unsqueeze for explicit last dim
X1_train = torch.tensor(
    [0.8750, 0.1500, 0.2500, 0.5000, 0.3853, 0.9991, 0.4333, 0.1494, 0.6196, 0.7808, 0.5609, 0.5895, 0.3395, 0.7232]).unsqueeze(-1)

X2_train = torch.tensor(
    [0.6750, 0.9750, 0.9000, 0.3500, 0.6010, 0.3451, 0.7451, 0.8499, 0.6240, 0.7218, 0.8389, 0.1664, 0.9771, 0.0598]).unsqueeze(-1)

X1_test = torch.tensor(X1_test.flatten()).squeeze().unsqueeze(-1)
X2_test = torch.tensor(X2_test.flatten()).squeeze().unsqueeze(-1)

# Hyperparameters

In [5]:
sigma_n = torch.tensor([0.05])

In [6]:
U_train, V_train = simulate_convergence(X1_train, X2_train)

U_train_noisy = U_train + sigma_n * torch.randn_like(U_train)
V_train_noisy = V_train + sigma_n * torch.randn_like(V_train)

# Concatenate along the first axis so that first half is Y1 and second half is Y2
# torch.Size([n_train x 2, 1])
Y_train_noisy = torch.cat((U_train_noisy, V_train_noisy), dim = 0)

In [7]:
X1_rows = X1_train
X2_rows = X2_train

X1_columns = X1_test
X2_columns = X2_test

In [8]:
X1_dist = (X1_rows.unsqueeze(1) - X1_columns.unsqueeze(0)).squeeze()
X2_dist = (X2_rows.unsqueeze(1) - X2_columns.unsqueeze(0)).squeeze()

X2_dist.shape

torch.Size([14, 441])

In [74]:
lengthscale = torch.tensor([0.3])
output_scalar = torch.tensor([1.0])

def divergence_free_se_kernel(X1_rows,
                              X2_rows, 
                              X1_columns, 
                              X2_columns, 
                              hyperparameters):
    
    # Extract hyperparameters
    sigma_f = hyperparameters[0]
    l = hyperparameters[1]
    
    # Calculate the (not quite Euclidean) distance between all pairs of points
    # This approach yields negative values as well
    # X1_dist: torch.Size([n_rows, n_columns])
    X1_dist = (X1_rows.unsqueeze(1) - X1_columns.unsqueeze(0)).squeeze()
    # ALTERNATIVE (pos.) torch.cdist(X1_rows, X1_columns): this is just torch.abs(X1_dist)

    # X2_dist: torch.Size([n_rows, n_columns])
    X2_dist = (X2_rows.unsqueeze(1) - X2_columns.unsqueeze(0)).squeeze()

    # torch.Size([n_rows, n_columns])
    upper_left = (1 - X2_dist.square().div(l**2)).div(l**2)

    # elementwise multiplication and division by scalar
    # Matlab version has negative values here! 
    upper_right = torch.mul(X1_dist, X2_dist).div(l**4)
    lower_left = upper_right
    lower_right = (1 - X1_dist.square().div(l**2)).div(l**2)

    # Concatenate upper and lower blocks column-wise, and then concatenate them row-wise
    # torch.Size([2 * n_train, 2 * n_test])
    block = torch.cat((torch.cat((upper_left, upper_right), 1), torch.cat((lower_left, lower_right), 1)), 0)

    # torch.Size([2 * n_train, 2 * n_test])
    # elementwise multiplication
    K = sigma_f.square() * block.mul((X1_dist.square() + X2_dist.square()).div(-2 * l**2).exp().tile(2, 2))

    return K

def predict(X1_train,
            X2_train,
            Y_train_noisy,
            X1_test, 
            X2_test, 
            hyperparameters,
            divergence_free_bool = True):
    
    # Extract N_x
    N_x_test = X1_test.shape[0]
    
    kernel_func = divergence_free_se_kernel if divergence_free_bool else block_diagonal_se_kernel

    K_train_train = kernel_func(
        X1_train, 
        X2_train, 
        X1_train, 
        X2_train,
        hyperparameters)

    # Add noise to the diagonal
    K_train_train_noisy = K_train_train + torch.eye(K_train_train.shape[0]) * sigma_n**2

    # torch.Size([2 * n_train, 2 * n_test])
    # K_* in Rasmussen is (X_train, X_test)
    K_train_test = kernel_func(
        X1_train, 
        X2_train, 
        X1_test, 
        X2_test,
        hyperparameters)

    # torch.Size([2 * n_test, 2 * n_train])
    K_test_train = K_train_test.mT

    K_test_test = kernel_func(
        X1_test, 
        X2_test, 
        X1_test, 
        X2_test,
        hyperparameters)
    
    # Determine L - torch.Size([2 * n_train, 2 * n_train])
    L = torch.linalg.cholesky(K_train_train_noisy, upper = False)
    # L.T \ (L \ y) in one step - torch.Size([2 * n_train, 1])
    alpha = torch.cholesky_solve(Y_train_noisy, L, upper = False)

    # matrix multiplication
    # torch.Size([2 * n_test, 2 * n_train]) * torch.Size([2 * n_train, 1])
    # alpha needs to be changes to double because K is
    predictive_mean = torch.matmul(K_test_train, alpha.double())
    predictive_mean_y1 = predictive_mean[: len(predictive_mean) // 2].reshape(N_x_test, N_x_test)
    predictive_mean_y2 = predictive_mean[len(predictive_mean) // 2 :].reshape(N_x_test, N_x_test)

    # Step 3: Solve for V = L^-1 * K(X_*, X)
    # K_* is K_train_test
    # L is lower triangular
    v = torch.linalg.solve_triangular(L, K_train_test, upper = False)
    # same as
    # v = torch.linalg.solve(L, K_train_test)
    # torch.matmul(v, v.T) would give the wrong shape
    predictive_covariance = K_test_test - torch.matmul(v.T, v)

    # Extract variance from diagonal for either dim and reshape into square grid
    predictive_variance_y1 = torch.diagonal(predictive_covariance)[:int(predictive_covariance.shape[0]/2)].reshape(N_x_test, N_x_test)
    predictive_variance_y2 = torch.diagonal(predictive_covariance)[int(predictive_covariance.shape[0]/2):].reshape(N_x_test, N_x_test)

    # matmul: torch.Size([1, 10]) * torch.Size([10, 1])
    # 0.5 * y^T * alpha
    # squeeze to remove redundant dimension
    nlml_term1 = 0.5 * torch.matmul(Y_train_noisy.T, alpha).squeeze()

    # sum(log(L_ii)))
    nlml_term2 = torch.sum(torch.log(torch.diagonal(L)))

    # Constant term - technically not optimised 
    # n/2 * log(2 * pi)
    nlml_term3 = (Y_train_noisy.shape[0]/2) * torch.log(torch.tensor(2 * math.pi))

    nlml = - nlml_term1 - nlml_term2 - nlml_term3

    # return predictive_mean_y1, predictive_mean_y2, predictive_variance_y1, predictive_variance_y2, nlml
    return predictive_mean_y1, predictive_mean_y2, predictive_variance_y1, predictive_variance_y2, nlml

In [91]:
scaled_diff[:, :, 0].shape
X1_dist.shape

torch.Size([14, 441, 1])

In [93]:
# In example, train are the rows

X1_dist = (X1_rows.unsqueeze(1) - X1_columns.unsqueeze(0))

rows = torch.cat((X1_rows, X2_rows), 1)[:, None, :]
# [1, n_columns, 2]
columns = torch.cat((X1_columns, X2_columns), 1)[None, :, :] 

scaled_diff = (rows - columns)

(scaled_diff[:, :, 0] == X1_dist.squeeze()).all()

tensor(True)

In [99]:
scaled_diff.square().sum(dim = -1).shape

torch.Size([14, 441])

In [ ]:
def divergence_free_se_kernel(
        row_tensor, # torch.Size([n_rows, 2])
        column_tensor, # torch.Size([n_columns, 2])
        hyperparameters):
    
    """
    Calculate the divergence-free SE kernel for two sets of points in 2D space.
    R^2 -> R^2

    Inputs:
        row_tensor: torch.Size([n_rows, 2])
        column_tensor: torch.Size([n_columns, 2])
        hyperparameters: list of length 2 containing sigma_f and l

    Returns:
        K: torch.Size([n_rows * 2, n_columns * 2])
    """
    
    # We calculate the kernel for each pair of points
    
    # Extract hyperparameters
    sigma_f = hyperparameters[0]
    l = hyperparameters[1]

    # Add dimension (broadcasting) for difference calculation
    # torch.Size([n_rows, 1, 2]) - 1 is for n_columns
    row_tensor_expanded = row_tensor[:, None, :]
    # torch.Size([1, n_columns, 2]) - 1 is for n_rowns
    column_tensor_expanded = column_tensor[None, :, :]

    # [:, :, 0] are the x1 differences and [:, :, 1] are the x2 differences
    # yields negative values as well
    diff = row_tensor_expanded - column_tensor_expanded

    ### 2x2 BLOCKS ###
    # x2 diffs: torch.Size([n_rows, n_columns])
    upper_left = (1 - diff[:, :, 1].square().div(l.square())).div(l.square())

    # x1 diffs: torch.Size([n_rows, n_columns])
    lower_right = (1 - diff[:, :, 0].square().div(l.square())).div(l.square())

    # Elementwise multiplication of x1 and x2 diffs and division by scalar
    # Matlab version has negative values here!
    upper_right = torch.prod(diff, dim = -1).div(l**4)

    # same as other off-diagonal block
    lower_left = upper_right

    # x1 diffs: torch.Size([n_rows, n_columns])
    lower_right = (1 - diff[:, :, 0].square().div(l.square())).div(l.square())

    # Concatenate upper and lower blocks column-wise, and then concatenate them row-wise
    # torch.Size([2 * n_train, 2 * n_test])
    blocks = torch.cat((
        torch.cat((upper_left, upper_right), 1), 
        torch.cat((lower_left, lower_right), 1)
        ), 0)

    # torch.Size([2 * n_row, 2 * n_column])
    # elementwise multiplication
    # sum squared difference over x1 and x2, divide by -2 * l^2, and exponentiate. Tile for blocks
    K = sigma_f.square() * blocks.mul(diff.square().sum(dim = -1).div(-2 * l.square()).exp().tile(2, 2))

    return K

In [55]:
X1_rows

# reshape to [n_rows, 2]
rows = torch.cat((X1_rows, X2_rows), 1)[:, None, :]
# [1, n_columns, 2]
columns = torch.cat((X1_columns, X2_columns), 1)[None, :, :] 

scaled_diff = (rows - columns)
sqdist = torch.sum(scaled_diff ** 2, dim = -1)

K_SE = torch.exp(-0.5 * sqdist)

B = torch.tensor([[1.0, 0.0], [0.0, 1.0]])

torch.kron(B, K_SE)

tensor([[0.5430, 0.5666, 0.5897,  ..., 0.0000, 0.0000, 0.0000],
        [0.6147, 0.6186, 0.6209,  ..., 0.0000, 0.0000, 0.0000],
        [0.6465, 0.6538, 0.6595,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.6732, 0.6620, 0.6494],
        [0.0000, 0.0000, 0.0000,  ..., 0.8544, 0.8298, 0.8038],
        [0.0000, 0.0000, 0.0000,  ..., 0.6328, 0.6264, 0.6186]],
       dtype=torch.float64)

In [ ]:
def block_diagonal_se_kernel(
        X1_rows,
        X2_rows, 
        X1_columns, 
        X2_columns, 
        hyperparameters):
    
    # block diagonal R^2 -> R^2
    # correlations between outputs are not considered
    
    # Extract hyperparameters
    sigma_f = hyperparameters[0]
    l = hyperparameters[1]
    B = hyperparameters[2]

    # reshape to [n_rows, 2] and then add a dimension [n_rows, 1, 2]
    rows = torch.cat((X1_rows, X2_rows), 1)[:, None, :]
    # [1, n_columns, 2]
    columns = torch.cat((X1_columns, X2_columns), 1)[None, :, :] 

    # difference scaled by lengthscales (2D)
    # torch.Size([n_rows, n_columns, 2])
    # Mahalanobis Distance (scaled)
    scaled_diff = (rows - columns) / l**2

    # square and reduce dimensions
    # torch.Size([n_rows, n_columns])
    sqdist = torch.sum(scaled_diff ** 2, dim = -1)

    # only one signal variance
    K_SE = sigma_f**2 * torch.exp(-0.5 * sqdist)
    
    # B is 2 x 2 and defines the cross correlation
    # Kronecker produces block diagonal
    K = torch.kron(B, K_SE)

    return K

In [100]:
# Define initial hyperparameters
hyperparameters_df = torch.tensor([
    torch.tensor([0.05]), 
    torch.tensor([0.6])], 
    requires_grad = True)

In [101]:
torch.cat((X1_train, X2_train), 1)

tensor([[0.8750, 0.6750],
        [0.1500, 0.9750],
        [0.2500, 0.9000],
        [0.5000, 0.3500],
        [0.3853, 0.6010],
        [0.9991, 0.3451],
        [0.4333, 0.7451],
        [0.1494, 0.8499],
        [0.6196, 0.6240],
        [0.7808, 0.7218],
        [0.5609, 0.8389],
        [0.5895, 0.1664],
        [0.3395, 0.9771],
        [0.7232, 0.0598]])

In [62]:
sigma_f = torch.tensor([0.05], requires_grad = True)
l = torch.tensor([0.6], requires_grad = True)

# list of tensors
hyperparameters_df = [sigma_f, l]

In [72]:
sigma_f = torch.tensor([0.05], requires_grad = True)
l = torch.tensor([0.6, 0.6], requires_grad = True)
B = torch.tensor([[1.0, 0.0], [0.0, 1.0]], requires_grad = True)

hyperparameters_bd = [sigma_f, l, B]

In [63]:
predictive_mean_y1, predictive_mean_y2, predictive_variance_y1, predictive_variance_y2, nlml = predict(
    X1_train,
    X2_train,
    Y_train_noisy,
    X1_test, 
    X2_test, 
    hyperparameters_df,
    divergence_free_bool = True)

In [76]:
predictive_mean_y1, predictive_mean_y2, predictive_variance_y1, predictive_variance_y2, nlml = predict(
    X1_train,
    X2_train,
    Y_train_noisy,
    X1_test, 
    X2_test, 
    hyperparameters_bd,
    divergence_free_bool = False)

RuntimeError: shape '[441, 441]' is invalid for input of size 441

In [70]:
optimizer = optim.Adam([sigma_f, l], lr = 0.01)

# Optimization loop
for i in range(100):  # Number of iterations
    optimizer.zero_grad()  # Reset gradients

    # Compute nlml
    _, _, _, _, nlml = predict(
        X1_train,
        X2_train,
        Y_train_noisy,
        X1_test, 
        X2_test, 
        hyperparameters_df,
        divergence_free_bool = True) # Compute NLML

    # Assign as loss 
    loss = nlml
    loss.backward()  # Compute gradients

    optimizer.step()  # Update hypers

    if i % 10 == 0:  # Print every 10 iterations
        print(f"Iter {i}: NLML = {loss.item()}, Hypers = {sigma_f.detach().numpy()[0], l.detach().numpy()[0]}")

print(f"Final Hypers: {sigma_f.detach().numpy()[0], l.detach().numpy()[0]}")

Iter 0: NLML = -2186.07177734375, Hypers = (0.009983333, 1.303558)
Iter 10: NLML = -2114.39599609375, Hypers = (0.0017196932, 1.3740392)
Iter 20: NLML = -2151.230224609375, Hypers = (0.0021803058, 1.4258841)
Iter 30: NLML = -2176.03955078125, Hypers = (0.0016361853, 1.4572905)
Iter 40: NLML = -2183.72900390625, Hypers = (0.0009911356, 1.4742708)
Iter 50: NLML = -2185.247802734375, Hypers = (0.00060369703, 1.4827492)
Iter 60: NLML = -2185.60986328125, Hypers = (0.00036646012, 1.4867553)
Iter 70: NLML = -2185.84765625, Hypers = (0.00016836595, 1.4885722)
Iter 80: NLML = -2186.035888671875, Hypers = (2.7742935e-06, 1.4893696)
Iter 90: NLML = -2186.07080078125, Hypers = (-7.0177135e-05, 1.4897105)
Final Hypers: (-5.102157e-05, 1.4898444)


In [37]:
hyperparameters.detach().numpy()

array([0.00005945, 0.8900958 ], dtype=float32)